# Hosting on k8s

A containerized version of the documentation is hosted on the NCAR | CISL k8s cluster and can be viewed at [https://cloud-docs.k8s.ucar.edu/]. 

## Updating the image

By default any changes pushed to the `docs/_build/html/` directory will trigger a GitHub action that will build a new image, named `docs:latest`, and push it to the cislcloudpilot Docker Hub repository. You should test any changes to this image locally before making changes to the hosted version with the following command: 

```
docker run --pull always -p 80:80 cislcloudpilot/docs:latest 
```

```{note}
The flag `--pull` has been set to always to ensure the latest version on Docker Hub is being run
```

If the changes look good it's time to push a new image tag to the container registry. This can be done 1 of 2 ways.

```{note}
Right now a production version of the image is tagged with the date it was released in the format month.day.year such as `Aug.8.23`.  
```

### Locally tag and push

```{note}
In order to successfully push the image to Docker Hub you will need to login with the following command : `docker login --username=cislcloudpilot`. If you do not have the password contact Nick Cote to get access to the KeePass file that contains it.
```

Since we just pulled down the `:latest` image to test our changes we can change the tag locally and push the new image tag back to the container registry. An example of how to do this is as follows:

```
docker image tag cislcloutpilot/docs:latest cislcloudpilot/docs:Aug.8.23
docker image push cislcloudpilot/docs:Aug.8.23
```

### Update GitHub workflow file

```{note}
The GitHub workflow file should be changed and pushed to GitHub ***before, or at the same time,*** as final changes made to `docs/_build/html/` are pushed to make sure the released image tag is applied when the Action is triggered by the change to `docs/_build/html/`.  
```

The GitHub workflow file found [here](https://github.com/NCAR/cisl-cloud/blob/main/.github/workflows/build-push-docs.yaml) defines the tags to push to Docker Hub. As of the writing of this document it's accomplished on line #48 with line #49 providing the default value : `cislcloudpilot/docs:latest`. Additional lines can be added to provide additional tags. Update the yaml file to include the new tag under line #48 and push the changes to GitHub making sure to take in to account the note above. Check the status of the new image build and push at the GitHub actions page [here](https://github.com/NCAR/cisl-cloud/actions).  

## Upgrade Helm

```{note}
This document assumes you have a kubeconfig file for the cluster and it is active. Documentation on how to accomplish this can be found [here](../K8s/Rancher/get-kubeconfig).
```

The cloud documentation was deployed on k8s using a helm chart. Instructions on how to accomplish this in more detail can be found [here](../K8s/Hosting/web-helm). Once you have a new docs image built, tagged, and pushed to Docker Hub it's time to update the helm chart hosted [here](https://github.com/NCAR/cisl-cloud/tree/main/configs/helm-charts/cloud-docs). More specifically we are going to change the `webapp.container.image` value that is set in the `values.yml` file and update the release information in `Chart.yaml`.

```{note}
The `appVersion` in `Chart.yaml` should directly match the image tag used in the container image. The proper `version` value should be incremented by 1 knowing that `version` is set to major.minor.patch. New content would be a change to the minor version number while any fixes that are implemented would be a change in the patch version number. The major version only changes when a very large change is made that ultimately affects how you interact with the image. 
```

With those changes in place to the helm chart files it's time to update the application on k8s. This is accomplished by running the following command in the `helm-charts` directory :

```
helm upgrade -n ncote-test cloud-docs cloud-docs/
```

If everything runs successfully you should see output that looks like the following:
```
Release "cloud-docs" has been upgraded. Happy Helming!
NAME: cloud-docs
LAST DEPLOYED: Wed Aug  9 12:06:41 2023
NAMESPACE: ncote-test
STATUS: deployed
REVISION: 2
TEST SUITE: None
```

If there's an issue Helm should let you know what it is. Please address it and run the upgrade command again until it succeeds. If you have issues that you are unable to overcome please reach out to a team member for assistance. 